**SM339 &#x25aa; Applied Statistics &#x25aa; Spring 2024 &#x25aa; Uhan**

# Lesson 22. Testing Subsets of Predictors &mdash; Part 2

### Example 1

Let's return to the `Perch` dataset. 

This dataset contains the weight (in grams), length (in centimeters), and width (in centimeters) for 56 perch caught in a lake in Finland.
Recall that we wanted to predict $\mathit{Weight}$ from $\mathit{Length}$ and $\mathit{Width}$.

In [1]:
library(Stat2Data)
data(Perch)
head(Perch)

,Obs,Weight,Length,Width
,<int>,<dbl>,<dbl>,<dbl>
1,104,5.9,8.8,1.4
2,105,32.0,14.7,2.0
3,106,40.0,16.0,2.4
4,107,51.5,17.2,2.6
5,108,70.0,18.5,2.9
6,109,100.0,19.2,3.3


- We considered three models, but two of them in particular were in close contention:

    1. Model 2, with both linear terms and an interaction term:
        $$ \text{Model 2:} \qquad \mathit{Weight} = \beta_0 + \beta_1 \mathit{Width} + \beta_2 \mathit{Length} + \beta_3 (\mathit{Width} \times \mathit{Length}) + \varepsilon \qquad \varepsilon \sim N(0, \sigma_{\varepsilon}^2) $$

    2. Model 3, the complete second-order model:
        $$ \text{Model 3:} \qquad \mathit{Weight} = \beta_0 + \beta_1 \mathit{Width} + \beta_2 \mathit{Length} + \beta_3 (\mathit{Width} \times \mathit{Length}) + \beta_4 \mathit{Width}^2 + \beta_5 \mathit{Length}^2 \qquad \varepsilon \sim N(0, \sigma_{\varepsilon}^2) $$

- Based on adjusted $R^2$ values and consdering parsimony, Model 2 was better than Model 3

- Now, we will _formally_ compare these two models, and see if the results support our informal argument

### a.

Let's conduct a __nested $F$-test__ to decide if we should remove _both_ the quadratic terms from the complete second-order model (Model 3)

- Recall that the hypotheses of the nested $F$-test in general are:

\begin{align*}
H_0: \beta_i = 0 & \quad \text{for all predictors in the subset} \\
H_A: \beta_i \ne 0 & \quad \text{for at least one predictor in the subset}
\end{align*}

- In this example, the subset of predictors we are testing is $\{ \mathit{Width}^2, \mathit{Length}^2 \}$

- So, the full model is Model 3, and the reduced model is Model 2

- The hypotheses of the nested $F$-test are:

\begin{align*}
& H_0: \beta_4 = \beta_5 = 0 \\
& H_A: \beta_4 \ne 0 \text{ and/or } \beta_5 \ne 0
\end{align*}

### b.
Compute the SSE for the full model.

In [2]:
# Solution
fit.full <- lm(Weight ~ Width + Length + Width:Length + I(Width^2) + I(Length^2), data = Perch)
summary(fit.full)

y <- Perch$Weight
SSE.full <- sum( (y - predict(fit.full))^2 )
SSE.full


Call:
lm(formula = Weight ~ Width + Length + Width:Length + I(Width^2) + 
    I(Length^2), data = Perch)

Residuals:
     Min       1Q   Median       3Q      Max 
-117.175  -11.904    2.822   11.556  157.596 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept)  156.3486    61.4152   2.546   0.0140 *
Width         20.9772    82.5877   0.254   0.8005  
Length       -25.0007    14.2729  -1.752   0.0860 .
I(Width^2)    34.4058    18.7455   1.835   0.0724 .
I(Length^2)    1.5719     0.7244   2.170   0.0348 *
Width:Length  -9.7763     7.1455  -1.368   0.1774  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 43.13 on 50 degrees of freedom
Multiple R-squared:  0.986,	Adjusted R-squared:  0.9846 
F-statistic: 704.6 on 5 and 50 DF,  p-value: < 2.2e-16


[1] 93000.1

### c.
Compute the SSE for the reduced model.

In [3]:
# Solution
fit.reduced <- lm(Weight ~ Width + Length + Width:Length, data = Perch)
summary(fit.reduced)

y <- Perch$Weight
SSE.reduced <- sum( (y - predict(fit.reduced))^2 )
SSE.reduced


Call:
lm(formula = Weight ~ Width + Length + Width:Length, data = Perch)

Residuals:
     Min       1Q   Median       3Q      Max 
-140.106  -12.226    1.230    8.489  181.408 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  113.9349    58.7844   1.938    0.058 .  
Width        -94.6309    22.2954  -4.244 9.06e-05 ***
Length        -3.4827     3.1521  -1.105    0.274    
Width:Length   5.2412     0.4131  12.687  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 44.24 on 52 degrees of freedom
Multiple R-squared:  0.9847,	Adjusted R-squared:  0.9838 
F-statistic:  1115 on 3 and 52 DF,  p-value: < 2.2e-16


[1] 101764.7

### d.

Calculate the test statistic:

$$ F = \frac{ (\mathit{SSE}_{\mathit{reduced}} - \mathit{SSE}_{\mathit{full}}) \,/\, \ell}{ \mathit{SSE}_{\mathit{full}} \,/\, (n - (k + 1)) } $$

In [4]:
# Solution
# n = number of observations = 56
# l = number of predictors being tested = 2
# k = number of predictors in full model = 5
F <- ((SSE.reduced - SSE.full) / 2) / (SSE.full / (56 - (5 + 1)))
F

[1] 2.356063

### e.
Calculate the p-value.

In [5]:
# Solution
1 - pf(F, df1 = 2, df2 = 50)

[1] 0.105235

### f.
What is your conclusion?

*Write your notes here. Double-click to edit.*

*Solution.* 
At a significance level of 0.05, we fail to reject the null hypothesis. 
We do not see evidence that including the two quadratic terms provides a significant improvement. We should use the reduced model (Model 2). 

### g.

As you may have guessed, R has a function that can perform all the steps of this nested F-test:

In [6]:
anova(fit.reduced, fit.full)    # the nested models to compare

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,52,101764.7,NA,NA,NA,NA
2,50,93000.1,2,8764.564,2.356063,0.105235
